Claribel Gonell

EN 705.603.82 - Module 6 - Assignment 6


# Directions

Create two notebooks:neo4jdating.ipynb and mongodating.ipynb.  Place all the answers in the notebooks with the final section in the mongodating.ipynb with the contrast observations between the two.  Although you are free to do it, it is not required to create a working docker image and place it into you docker hub repository.

**Import main mongo class**

In [1]:
from pymongo import MongoClient

**Connect to mongo database**

In [2]:
client = MongoClient('localhost', 27017)

**1) List current databases in mongo**

In [3]:
dbs = client.list_database_names()
print(dbs)

['admin', 'config', 'local', 'mydatabase', 'speeddating']


**2) Use speeddating databse**

In [4]:
db = client['speeddating']

**3) Show collections**

In [5]:
collections = db.list_collection_names()
print(collections)

['events']


**4) Show one of the collection documents**

In [6]:
find_one = db.events.find_one()


**5) Find all matches**

In [7]:
all_matches = db.events.find({"match":1})



**Now query the features you need to build a prediction model for matches**

In [8]:
from pandas import DataFrame
import pandas as pd
import numpy as np
features = db.events.find( { }, { "match": 1,"intelligence": 1, "intelligence_o": 1,"attractive": 1,
                                 "attractive_o": 1,"funny": 1,
                                "funny_o": 1, })
list_feat = list(features)
df = DataFrame(list_feat)
df = df.drop(columns=['_id'])
print(df.dtypes)
print(df.head)

# Convert numeric function string to float

df['attractive_o'] = pd.to_numeric(df['attractive_o'], errors='coerce')
df['intelligence'] = pd.to_numeric(df['intelligence'], errors='coerce')
df['intelligence_o'] = pd.to_numeric(df['intelligence_o'], errors='coerce')
df['attractive'] = pd.to_numeric(df['attractive'], errors='coerce')
df['funny'] = pd.to_numeric(df['funny'], errors='coerce')
df['funny_o'] = pd.to_numeric(df['funny_o'], errors='coerce')

df = df.replace(np.nan, 0, regex=True)

print(df)
print(df.dtypes)


attractive_o      object
intelligence_o    object
funny_o           object
attractive        object
intelligence      object
funny             object
match              int64
dtype: object
<bound method NDFrame.head of      attractive_o intelligence_o funny_o attractive intelligence funny  match
0               7             10       7          6            8     8      0
1              10             10      10          6            8     8      1
2               7              9       8          6            8     8      1
3               6              8       8          6            8     8      0
4               8              9       6          6            8     8      1
...           ...            ...     ...        ...          ...   ...    ...
8373            8              7       7          8            6     7      0
8374            6              7       3          8            6     7      0
8375            2              2       2          8            6     7      0
8

In [9]:
import warnings
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import KFold
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KernelDensity
from sklearn.svm import SVC


In [10]:
y = df['match']
X = df.drop(columns=['match'])


In [11]:
# Implementing cross validation
# Instantiating the K-Fold cross validation object with 5 folds
k_folds = KFold(n_splits=5, shuffle=True)
# Iterating through each of the folds in K-Fold
for train_index, val_index in k_folds.split(X):
    print("Train :", train_index, "Test: ", val_index)
    X_train, X_val = X.iloc[train_index, :], X.iloc[val_index, :]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]

    # Instantiating a RandomForestClassifier model
    rfc_model = RandomForestClassifier()
    svc_model = SVC(kernel='linear')  # Support Vector Machine
    gnb_model = GaussianNB()  # Gaussian

    # Fitting the X_train and y_train datasets to models below
    rfc_model.fit(X_train, y_train)
    svc_model.fit(X_train, y_train)
    gnb_model.fit(X_train, y_train)

    # predictions for vaildation sets
    val_preds_rfc = rfc_model.predict(X_val)

    val_preds_svc = svc_model.predict(X_val)
    val_preds_gnb = gnb_model.predict(X_val)

    # Validation Metrics 
    val_accuracy_rfc = accuracy_score(y_val, val_preds_rfc)
    val_confusion_matrix_rfc = confusion_matrix(y_val, val_preds_rfc)

    val_accuracy_svc = accuracy_score(y_val, val_preds_svc)
    val_confusion_matrix_svc = confusion_matrix(y_val, val_preds_svc)

    val_accuracy_gnb = accuracy_score(y_val, val_preds_gnb)
    val_confusion_matrix_gnb = confusion_matrix(y_val, val_preds_gnb)

    # Printing out the validation metrics
    print(f'Accuracy Score Random Forest: {val_accuracy_rfc}')
    print(f'Accuracy Score SVC: {val_accuracy_svc}')
    print(f'Accuracy Score GaussianNB: {val_accuracy_gnb}')

Train : [   1    2    3 ... 8375 8376 8377] Test:  [   0    6    9 ... 8357 8358 8370]
Accuracy Score Random Forest: 0.788782816229117
Accuracy Score SVC: 0.8251789976133651
Accuracy Score GaussianNB: 0.7804295942720764
Train : [   0    1    2 ... 8373 8376 8377] Test:  [   4    7   10 ... 8366 8374 8375]
Accuracy Score Random Forest: 0.8174224343675418
Accuracy Score SVC: 0.8466587112171837
Accuracy Score GaussianNB: 0.795346062052506
Train : [   0    1    2 ... 8374 8375 8377] Test:  [   3   12   22 ... 8368 8373 8376]
Accuracy Score Random Forest: 0.7875894988066826
Accuracy Score SVC: 0.8174224343675418
Accuracy Score GaussianNB: 0.7834128878281623
Train : [   0    1    2 ... 8374 8375 8376] Test:  [   5   14   18 ... 8371 8372 8377]
Accuracy Score Random Forest: 0.8214925373134329
Accuracy Score SVC: 0.8483582089552238
Accuracy Score GaussianNB: 0.786865671641791
Train : [   0    3    4 ... 8375 8376 8377] Test:  [   1    2    8 ... 8355 8356 8367]
Accuracy Score Random Forest: 0.

**Finally, at the end of the notebook enter a section that contrast the two databases.  Note the advantages and disadvantages of each?  What type of situation is each best suited for?**

**Neo4J**

Advantages: schema optional, navigation through tree traversal(very true to graph structure)

Disadvantages: no partioning methods

Best Suitable For: Graph database management systems; data that requires entity-relationship integrity

**MongoDB**

Advantages: document store structure makes it very easy to query and is schema-less, very flexible

Disadvantages: cannot create relationships 

Best Suitable For: Everything else. 